In [ ]:
%pip install tensorflow

In [ ]:
%pip install matplotlib

In [ ]:
%pip install tensorflow-datasets
%pip install ipywidgets

In [ ]:
%pip install --upgrade pip setuptools

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
for gpu in gpus:
    print(gpu)

In [ ]:
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

In [ ]:
first_batch = next(ds.as_numpy_iterator())
print(first_batch.keys())  # Should output: dict_keys(['image'])


In [ ]:
import numpy as np
dataiterator = ds.as_numpy_iterator() # set up a connection to the dataset

In [ ]:
np.squeeze(dataiterator.next()['image']).shape  # get the next batch of data

In [ ]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import numpy as np

# Path to your custom dataset
CUSTOM_DATASET_DIR = "C:/Users/HP/Documents/Programs/Cynaptics/Data/Train/Real"

# Function to preprocess and load images
def load_and_preprocess_image(image_path, target_size=(28, 28)):
    image_path = image_path.numpy().decode('utf-8')  # Convert TensorFlow tensor to Python string
    image = load_img(image_path, target_size=target_size, color_mode="grayscale")  # Fashion MNIST is grayscale
    image = img_to_array(image) / 255.0  # Normalize to [0, 1]
    return image

# Wrapper to handle `numpy()` conversion in the map function
def load_and_preprocess_image_wrapper(image_path, target_size=(28, 28)):
    return tf.py_function(func=load_and_preprocess_image, inp=[image_path], Tout=tf.float32)

# Function to create a TensorFlow Dataset
def create_custom_dataset(directory, target_size=(28, 28), batch_size=32):
    image_paths = [os.path.join(directory, fname) for fname in os.listdir(directory) if fname.endswith(('.png', '.jpg', '.jpeg'))]
    labels = [0] * len(image_paths)  # Dummy labels, as the dataset has no explicit labels
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(lambda x, y: {'image': load_and_preprocess_image_wrapper(x, target_size), 'label': y}, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=len(image_paths)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Create the custom dataset
ds = create_custom_dataset(CUSTOM_DATASET_DIR)

# Visualization of Images with Matplotlib
dataiterator = iter(ds)  # Create an iterator for the dataset

# Set up the sub-plot format
fig, ax = plt.subplots(ncols=4, figsize=(20, 20))  # fig is the whole plot, ax is each subplot
for idx in range(4):
    # Grab an image and label
    batch = next(dataiterator)  # A sample
    image = np.squeeze(batch['image'][idx].numpy())  # Extract the image and remove batch dimensions
    label = batch['label'][idx].numpy()  # Extract the corresponding label
    # Plot the image using a specific axis
    ax[idx].imshow(image, cmap='gray')  # Display the grayscale image
    ax[idx].set_title(f"Label: {label}")  # Append image label as plot title
    ax[idx].axis('off')  # Hide axes for clarity

plt.show()


In [ ]:
# Data processing , scaling images to 0-1
def scale_images(data):
  image = data['image']
  return image/ 255

In [ ]:
# dataset
ds = tfds.load('fashion_mnist', split='train') # Reloaded dataset , optional
ds = ds.map(scale_images) # Running dataset through the scaling function
ds = ds.cache() # Caching the dataset for that batch
ds = ds.shuffle(60000) #Shuffling the dataset
ds = ds.batch(128) # Dividing the dataset into batches of 128
ds = ds.prefetch(64)  # Eliminates bottelneckind by prefetching the next batch

In [ ]:
#Bringing in sqquential api for generator and discriminator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, Reshape , Flatten, Conv2D, Dropout,UpSampling2D

In [ ]:
def build_generator(): 
    model = Sequential()
    
    # Takes in random values and reshapes it to 7x7x128
    # Beginnings of a generated image
    model.add(Dense(7*7*128, input_dim=128))
    model.add(LeakyReLU(0.2))
    model.add(Reshape((7,7,128)))
    
    # Upsampling block 1 
    model.add(UpSampling2D())
    model.add(Conv2D(128, 5, padding='same'))
    model.add(LeakyReLU(0.2))
    
    # Upsampling block 2 
    model.add(UpSampling2D())
    model.add(Conv2D(128, 5, padding='same'))
    model.add(LeakyReLU(0.2))
    
    # Convolutional block 1
    model.add(Conv2D(128, 4, padding='same'))
    model.add(LeakyReLU(0.2))
    
    # Convolutional block 2
    model.add(Conv2D(128, 4, padding='same'))
    model.add(LeakyReLU(0.2))
    
    # Conv layer to get to one channel
    model.add(Conv2D(1, 4, padding='same', activation='sigmoid'))
    
    return model


In [ ]:
generator = build_generator()
generator.summary()

In [ ]:
#Testing the generator
img = generator.predict(np.random.randn(4,128,1))
# Set up the sub-plot format

fig ,ax = plt.subplots(ncols = 4,figsize=(20,20)) # fig is whole plot , ax is each subplot
for idx , img in enumerate(img):

  #Plot the image using a spicific axis
  ax[idx].imshow(np.squeeze(img)) # plot using a specific subplot
  ax[idx].set_title(idx) #Appending image lable as plot title

In [ ]:
def build_discriminator():
  model = Sequential()
  
  #First Conv block
  model.add(Conv2D(32 , 5, input_shape=(28,28,1)))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))
  
  #Second Conv block
  model.add(Conv2D(64 , 5))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))
  
  #Third Conv block
  model.add(Conv2D(128 , 5))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))
  
  #Forth Conv block
  model.add(Conv2D(256 , 5))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))
  
  #Flatten then pass through dense layer
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(1, activation='sigmoid'))
  
  return model

In [ ]:
discriminator = build_discriminator()
discriminator.summary()

In [ ]:
discriminator.predict(np.expand_dims(img,0))

In [ ]:
# Importing the optimizer and loss function
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [ ]:
g_opt = Adam(learning_rate=0.0001)
d_opt = Adam(learning_rate=0.00001)
g_loss = BinaryCrossentropy()
d_loss = BinaryCrossentropy()

In [ ]:
#mporting base model class into sub-class
from tensorflow.keras.models import Model

In [ ]:
class GAN(Model): 
    def __init__(self, generator, discriminator, *args, **kwargs):
        # Pass through args and kwargs to base class 
        super().__init__(*args, **kwargs)
        
        # Create attributes for gen and disc
        self.generator = generator 
        self.discriminator = discriminator 
        
    def compile(self, g_opt, d_opt, g_loss, d_loss, *args, **kwargs): 
        # Compile with base class
        super().compile(*args, **kwargs)
        
        # Create attributes for losses and optimizers
        self.g_opt = g_opt
        self.d_opt = d_opt
        self.g_loss = g_loss
        self.d_loss = d_loss 

    def train_step(self, batch):
        # Get the data 
        real_images = batch
        fake_images = self.generator(tf.random.normal((128, 128, 1)), training=False)
        
        # Train the discriminator
        with tf.GradientTape() as d_tape: 
            # Pass the real and fake images to the discriminator model
            yhat_real = self.discriminator(real_images, training=True) 
            yhat_fake = self.discriminator(fake_images, training=True)
            yhat_realfake = tf.concat([yhat_real, yhat_fake], axis=0)
            
            # Create labels for real and fakes images
            y_realfake = tf.concat([tf.zeros_like(yhat_real), tf.ones_like(yhat_fake)], axis=0)
            
            # Add some noise to the TRUE outputs
            noise_real = 0.15*tf.random.uniform(tf.shape(yhat_real))
            noise_fake = -0.15*tf.random.uniform(tf.shape(yhat_fake))
            y_realfake += tf.concat([noise_real, noise_fake], axis=0)
            
            # Calculate loss - BINARYCROSS 
            total_d_loss = self.d_loss(y_realfake, yhat_realfake)
            
        # Apply backpropagation - nn learn 
        dgrad = d_tape.gradient(total_d_loss, self.discriminator.trainable_variables) 
        self.d_opt.apply_gradients(zip(dgrad, self.discriminator.trainable_variables))
        
        # Train the generator 
        with tf.GradientTape() as g_tape: 
            # Generate some new images
            gen_images = self.generator(tf.random.normal((128,128,1)), training=True)
                                        
            # Create the predicted labels
            predicted_labels = self.discriminator(gen_images, training=False)
                                        
            # Calculate loss - trick to training to fake out the discriminator
            total_g_loss = self.g_loss(tf.zeros_like(predicted_labels), predicted_labels) 
            
        # Apply backprop
        ggrad = g_tape.gradient(total_g_loss, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(ggrad, self.generator.trainable_variables))
        
        return {"d_loss":total_d_loss, "g_loss":total_g_loss}

In [ ]:
#Create instance of the subclass model
gan = GAN(generator , discriminator)
#Compile the model
gan.compile(g_opt , d_opt , g_loss , d_loss)

In [ ]:
import os
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import Callback

In [ ]:
class ModelMonitor(Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.uniform((self.num_img, self.latent_dim,1))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = array_to_img(generated_images[i])
            img.save(os.path.join('generated_images', f'generated_img_{epoch}_{i}.png'))

In [ ]:
hist = gan.fit(ds , epochs = 2000 ,callbacks = [ModelMonitor()])

In [ ]:
plt.suptitle('Loss')
plt.plot(hist.history['d_loss'], label='Discriminator Loss')
plt.plot(hist.history['g_loss'], label='Generator Loss')
plt.legend()
plt.show()

In [ ]:
imgs = generator(tf.random.normal((16 , 128 , 1)), training=False)

In [ ]:
fig , ax = plt.subplots(ncols = 4,nrows =4, figsize=(20,20))
for idx in range(4):
  for c in range(4):
    ax[idx][c].imshow(imgs[(idx+1)*(c+1)-1])